In [2]:
import os
import sys
sys.path.append('../')
import utils.csv as csv
import utils.plot as plot
from sklearn.metrics import accuracy_score

In [22]:
PATH='D:\\Deutschland\\FUB\\master_thesis\\data'
DATA_DIR = os.path.join(PATH, 'gee', 'output', 'validation_grid_daily_padding')
LABEL_CSV = 'label_aoi.csv'
METHOD = 'classification'
MODEL = 'transformer'
UID = '8pure9'
MODEL_NAME = MODEL + '_' + UID
LABEL_PATH = os.path.join(PATH, 'ref', 'validation', LABEL_CSV)
MODEL_PATH = f'../outputs/models/{METHOD}/02/{MODEL_NAME}.pth'

In [12]:
 # *************************change class here*************************
classes = ['Spruce', 'Sliver Fir','Douglas Fir','Pine','Oak','Red Oak','Beech','Sycamore','Others']
# *******************************************************************
ref = csv.load(f'../outputs/csv/{METHOD}/02/{MODEL_NAME}_ref.csv', 'id')
pred = csv.load(f'../outputs/csv/{METHOD}/02/{MODEL_NAME}_pred.csv', 'id')

In [13]:
accuracy_score(ref, pred)

0.8696985181400102

In [12]:
plot.draw_confusion_matrix(ref, pred, classes, MODEL_NAME)

<Figure size 1000x1000 with 0 Axes>

In [19]:
import geopandas as gpd
import utils.shp as shp
from geopandas.tools import sjoin
file_path = os.path.join(PATH, 'Reference_data', 'bw_all_polygons', 'buffered_wgs_bw_polygons.shp')
aoi_path = os.path.join(PATH, 'Reference_data', 'validation', 'Objectbased_classification_regression_validation2.shp')
out_path = os.path.join(PATH, 'Reference_data', 'validation', 'aoi_polygons.shp')

In [9]:
polygons = shp.load_shp_file(file_path)
polygons

import file D:\Deutschland\FUB\master_thesis\data\Reference_data\validation\Objectbased_classification_regression_validation2.shp
import file D:\Deutschland\FUB\master_thesis\data\Reference_data\bw_all_polygons\buffered_wgs_bw_polygons.shp


,BST1_BA_1,BST1_BA_2,BST1_BA_3,BST1_BA_4,BST1_BA_5,BST1_BA_6,BST1_BA_7,BST1_BA_8,BST1_BAA_1,BST1_BAA_2,...,BST3_BAA_1,BST3_BAA_2,BST3_BAA_3,BST3_BAA_4,BST3_BAA_5,BST3_BAA_6,BST3_BAA_7,BST3_BAA_8,id,geometry
0,110,943,920,979,0,0,0,0,100,0,...,0,0,0,0,0,0,0,0,1,"POLYGON ((9.71275 48.83270, 9.71276 48.83275, ..."
1,110,831,600,710,920,0,0,0,95,5,...,0,0,0,0,0,0,0,0,2,"POLYGON ((9.71103 48.82652, 9.71106 48.82653, ..."
2,110,831,820,921,710,979,0,0,60,20,...,0,0,0,0,0,0,0,0,3,"POLYGON ((9.71794 48.83711, 9.71794 48.83712, ..."
3,110,710,831,0,0,0,0,0,60,40,...,0,0,0,0,0,0,0,0,4,"POLYGON ((9.71427 48.82590, 9.71429 48.82593, ..."
4,110,831,710,821,930,600,210,0,40,30,...,0,0,0,0,0,0,0,0,5,"POLYGON ((9.71447 48.82624, 9.71446 48.82625, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283607,410,710,600,310,810,910,0,0,50,40,...,0,0,0,0,0,0,0,0,418137,"MULTIPOLYGON (((8.62672 49.34822, 8.62517 49.3..."
283608,410,710,600,910,810,0,0,0,70,20,...,0,0,0,0,0,0,0,0,418139,"POLYGON ((8.61916 49.35218, 8.61918 49.35221, ..."
283609,410,710,6,810,110,881,600,821,70,30,...,0,0,0,0,0,0,0,0,418141,"POLYGON ((8.61311 49.33944, 8.61375 49.33975, ..."
283610,710,831,410,600,810,110,822,630,45,20,...,0,0,0,0,0,0,0,0,418142,None


In [15]:
aoi = shp.load_shp_file(aoi_path)
aoi.drop(columns=['id'], inplace=True)
aoi = shp.reproject(aoi)
aoi

import file D:\Deutschland\FUB\master_thesis\data\Reference_data\validation\Objectbased_classification_regression_validation2.shp
Reproject to EPSG:4326


,Location,geometry
0,Hardtwald_pine_beech_redoak,"POLYGON ((8.42299 49.08763, 8.45038 49.08776, ..."
1,Schwarzwald_spruce_silverfir_douglasfir,"POLYGON ((7.90144 48.00939, 7.92825 48.00964, ..."
2,schoenbuch_beech_oak_mixture,"POLYGON ((8.93597 48.59956, 8.96309 48.59958, ..."


In [23]:
assert polygons.crs == aoi.crs, "Coordinate Reference Systems (CRS) must match"
val_polygons = sjoin(polygons, aoi, how='inner', op='intersects')
val_polygons.drop(columns=['index_right'], inplace=True)
shp.export_shp_file(val_polygons, out_path)
shp.export_csv_reference(val_polygons, LABEL_PATH)

d:\LenovoSoftstore\Anaconda\envs\yolov5\lib\site-packages\IPython\core\interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


export file D:\Deutschland\FUB\master_thesis\data\Reference_data\validation\aoi_polygons.shp
export file D:\Deutschland\FUB\master_thesis\data\ref\validation\label_aoi.csv
